In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/blending/train_oof.csv
/kaggle/input/blending/__results__.html
/kaggle/input/blending/__notebook_source__.ipynb
/kaggle/input/blending/__notebook__.ipynb
/kaggle/input/blending/__output__.json
/kaggle/input/blending/oof.csv
/kaggle/input/blending/test_oof.csv
/kaggle/input/blending/custom.css
/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2021/train.csv
/kaggle/input/tabular-playground-series-aug-2021/test.csv


In [3]:
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression, ElasticNet, BayesianRidge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
train_oof = pd.read_csv("/kaggle/input/blending/train_oof.csv", index_col="id")
test_oof  = pd.read_csv("/kaggle/input/blending/test_oof.csv", index_col="id")

train = pd.read_csv("/kaggle/input/tabular-playground-series-aug-2021/train.csv", index_col="id")
target = train["loss"]
train = train.drop("loss", axis=1)
test = pd.read_csv("/kaggle/input/tabular-playground-series-aug-2021/test.csv", index_col="id")

train_oof = train_oof.drop(["lr", "elst", "br"], axis=1)
test_oof = test_oof.drop(["lr", "elst", "br"], axis=1)

In [5]:
train = pd.concat([train, train_oof],axis=1)
test  = pd.concat([test , test_oof ],axis=1)

In [6]:
import optuna 
import optuna.integration.lightgbm as lgbo

params = {'objective': 'poisson',  'metric': 'rmse' } #'objective': 'mean_squared_error',
from sklearn.metrics import mean_absolute_error
x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.3, random_state=42)
lgb_train = lgb.Dataset(x_train, y_train)
lgb_valid = lgb.Dataset(x_test, y_test)
model = lgbo.train(params, lgb_train, valid_sets=[lgb_valid], verbose_eval=False, num_boost_round=100, early_stopping_rounds=5) 
model.params

[I 2021-08-25 21:51:27,181] A new study created in memory with name: no-name-b81eaf16-ecd6-49a1-ade2-b6bb8bbfd418
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012522 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 26263
[LightGBM] [Info] Number of data points in the train set: 175000, number of used features: 103
[LightGBM] [Info] Start training from score 1.920970


feature_fraction, val_score: 7.845767:  14%|#4        | 1/7 [00:04<00:25,  4.33s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.116877 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26263
[LightGBM] [Info] Number of data points in the train set: 175000, number of used features: 103
[LightGBM] [Info] Start training from score 1.920970


feature_fraction, val_score: 7.844309:  29%|##8       | 2/7 [00:10<00:26,  5.28s/it]

...

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.117039 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26263
[LightGBM] [Info] Number of data points in the train set: 175000, number of used features: 103
[LightGBM] [Info] Start training from score 1.920970


min_data_in_leaf, val_score: 7.843105:  80%|########  | 4/5 [00:17<00:04,  4.40s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.116101 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26263
[LightGBM] [Info] Number of data points in the train set: 175000, number of used features: 103
[LightGBM] [Info] Start training from score 1.920970


min_data_in_leaf, val_score: 7.843105: 100%|##########| 5/5 [00:21<00:00,  4.39s/it]


{'objective': 'poisson',
 'metric': 'rmse',
 'feature_pre_filter': False,
 'lambda_l1': 8.916988279459193,
 'lambda_l2': 1.7943343419272821,
 'num_leaves': 10,
 'feature_fraction': 0.9840000000000001,
 'bagging_fraction': 0.6664545812936506,
 'bagging_freq': 5,
 'min_child_samples': 20,
 'num_iterations': 100,
 'early_stopping_round': 5}

In [32]:
model_params = {'objective': 'poisson',
                 'metric': 'rmse',
                 'feature_pre_filter': False,
                 'lambda_l1': 8.916988279459193,
                 'lambda_l2': 1.7943343419272821,
                 'num_leaves': 10,
                 'feature_fraction': 0.9840000000000001,
                 'bagging_fraction': 0.6664545812936506,
                 'bagging_freq': 5,
                 'min_child_samples': 20,
                 'num_iterations': 100,
                 'early_stopping_round': 5,
               "learning_rate": 0.01,
               "num_iterations":80000,
               "early_stopping_round": 200}

In [33]:
final_preds = np.zeros(test.shape[0])
final_train_oof = np.zeros(train.shape[0])

kfd = KFold(n_splits=10, shuffle=True)

for fold, (train_idx, valid_idx) in enumerate(kfd.split(X=train, y=target)):
    print("-"*20, f"FOLD {fold}", "-"*20)
    X_train, X_valid = train.iloc[train_idx], train.iloc[valid_idx]
    y_train, y_valid = target.iloc[train_idx], target.iloc[valid_idx]
    
    train_dataset = lgb.Dataset(X_train, y_train)
    valid_dataset = lgb.Dataset(X_valid, y_valid)
    
    model = lgb.train(model_params, train_dataset, valid_sets=[valid_dataset], verbose_eval=1000)
    
    final_train_oof[valid_idx] = model.predict(X_valid)
    final_preds += model.predict(test) / 10

-------------------- FOLD 0 --------------------
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.138454 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26264
[LightGBM] [Info] Number of data points in the train set: 225000, number of used features: 103
[LightGBM] [Info] Start training from score 1.920243
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[660]	valid_0's rmse: 7.80335
-------------------- FOLD 1 --------------------
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.139654 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26263
[LightGBM] [Info] Number of data points in the train set: 225000, number of used features: 103
[LightGBM] [Info] Start training from score 1.919283
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's 

In [35]:
predictions = pd.DataFrame()
predictions["id"] = test.index
predictions["loss"] = final_preds

predictions.to_csv('final_submission4.csv', index=False, header=predictions.columns)

In [39]:
predictions = pd.DataFrame()
predictions["id"] = train.index
predictions["loss"] = final_train_oof

predictions.to_csv('final_train_oof.csv', index=False, header=predictions.columns)